# Lab Assignment 7: Database Queries
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

### Problem 0
Import the following libraries, load the `.env` file where you store your passwords (see the notebook for module 4 for details), and turn off the error tracebacks to make errors easier to read:

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import requests
import psycopg
import pymongo
import json
from bson.json_util import dumps, loads
from sqlalchemy import create_engine
import dotenv

# change to the directory where your .env file is
os.chdir("/Users/leeka/Downloads/ds6001databases") 

dotenv.load_dotenv() # register the .env file where passwords are stored
sys.tracebacklimit = 0 # turn off the error tracebacks

### Problem 1
For this problem, we will be building a PostgreSQL database that contains the collected works of Shakespeare.

<img src="https://www.chappatte.com/prod/wp-content/uploads/artworks/2016/04/L160423ge-950x635.jpg" width="300">

The data were collected by [Catherine Devlin](https://github.com/catherinedevlin/opensourceshakespeare) from the repository at https://opensourceshakespeare.org/. The database will have four tables, one representing works by Shakespeare, one for characters that appear in Shakespeare's plays, one for chapters (this is, scenes within acts), and one for paragraphs (that is, lines of dialogue). The data to populate these four tables are here: 

In [2]:
works = pd.read_csv("https://github.com/jkropko/DS-6001/raw/master/localdata/Works.csv")
characters = pd.read_csv("https://github.com/jkropko/DS-6001/raw/master/localdata/Characters.csv")
chapters = pd.read_csv("https://github.com/jkropko/DS-6001/raw/master/localdata/Chapters.csv")
paragraphs = pd.read_csv("https://github.com/jkropko/DS-6001/raw/master/localdata/Paragraphs.csv")

In PostgreSQL, it is best practice to convert all column names to lower-case, as case sensitive column names will require [extraneous double-quotes](https://stackoverflow.com/questions/20878932/are-postgresql-column-names-case-sensitive) in any query. We first convert the column names in all four dataframe to lowercase:

In [3]:
works.columns = works.columns.str.lower()
characters.columns = characters.columns.str.lower()
chapters.columns = chapters.columns.str.lower()
paragraphs.columns = paragraphs.columns.str.lower()

You will build a database and populate it with these data. The ER diagram for the database is:

<img src="https://github.com/jkropko/DS-6001/raw/master/localimages/shakespeare2.png" width="400">

There's no codebook, unfortunately, but the values in the columns are mostly self-explanatory:

In [4]:
works.head() 

,workid,title,longtitle,date,genretype,notes,source,totalwords,totalparagraphs
0,12night,Twelfth Night,"Twelfth Night, Or What You Will",1599,c,NaN,Moby,19837,1031
1,allswell,All's Well That Ends Well,All's Well That Ends Well,1602,c,NaN,Moby,22997,1025
2,antonycleo,Antony and Cleopatra,Antony and Cleopatra,1606,t,NaN,Moby,24905,1344
3,asyoulikeit,As You Like It,As You Like It,1599,c,NaN,Gutenberg,21690,872
4,comedyerrors,Comedy of Errors,The Comedy of Errors,1589,c,NaN,Moby,14692,661


In [5]:
characters.head()

,charid,charname,abbrev,works,description,speechcount
0,1apparition-mac,First Apparition,First Apparition,macbeth,NaN,1.0
1,1citizen,First Citizen,First Citizen,romeojuliet,NaN,3.0
2,1conspirator,First Conspirator,First Conspirator,coriolanus,NaN,3.0
3,1gentleman-oth,First Gentleman,First Gentleman,othello,NaN,1.0
4,1goth,First Goth,First Goth,titus,NaN,4.0


In [6]:
chapters.head()

,workid,chapterid,section,chapter,description
0,12night,18704.0,1.0,1.0,DUKE ORSINO's palace.
1,12night,18705.0,1.0,2.0,The sea-coast.
2,12night,18706.0,1.0,3.0,OLIVIA'S house.
3,12night,18707.0,1.0,4.0,DUKE ORSINO's palace.
4,12night,18708.0,1.0,5.0,OLIVIA'S house.


In [7]:
paragraphs.head()

,workid,paragraphid,paragraphnum,charid,plaintext,phonetictext,stemtext,paragraphtype,section,chapter,charcount,wordcount
0,12night,630863,3,xxx,"[Enter DUKE ORSINO, CURIO, and other Lords; Mu...",ENTR TK ORSN KR ANT O0R LRTS MSXNS ATNTNK,enter duke orsino curio and other lord musicia...,b,1.0,1.0,65.0,9.0
1,12night,630864,4,ORSINO,"If music be the food of love, play on;\n[p]Giv...",IF MSK B 0 FT OF LF PL ON JF M EKSSS OF IT 0T ...,if music be the food of love plai on give me e...,b,1.0,1.0,646.0,114.0
2,12night,630865,19,CURIO,"Will you go hunt, my lord?\n",WL Y K HNT M LRT,will you go hunt my lord,b,1.0,1.0,27.0,6.0
3,12night,630866,20,ORSINO,"What, Curio?\n",HT KR,what curio,b,1.0,1.0,13.0,2.0
4,12night,630867,21,CURIO,The hart.\n,0 HRT,the hart,b,1.0,1.0,10.0,2.0


#### Part a
Connect to your local PostgreSQL server (take steps to hide your password!), create a new database for the Shakespeare data, use `create_engine()` from `sqlalchemy` to connect to the database, and create the works, characters, chapters, and paragraphs tables populated with the data from the four dataframes shown above. [2 points]

In [8]:
# Environmental variables
dotenv.load_dotenv()
postgres_password = os.getenv('POSTGRES_PASSWORD')

# Connect to postgres server
dbserver = psycopg.connect(
    user = 'postgres',
    password = postgres_password,
    host = 'localhost',
    port = '5432'
)

dbserver.autocommit = True

OperationalError: connection failed: connection to server at "127.0.0.1", port 5432 failed: could not receive data from server: Socket is not connected (0x00002749/10057)
could not send SSL negotiation packet: Socket is not connected (0x00002749/10057)

In [ ]:
# Create shakespeare database
cursor = dbserver.cursor()

try:
    cursor.execute('CREATE DATABASE shakespeare')
except:
    cursor.execute('DROP DATABASE shakespeare')
    cursor.execute('CREATE DATABASE shakespeare')

In [ ]:
# Upload dataframes to NBA database
dbms = 'postgresql'
connector = 'psycopg'
user = 'postgres'
password = postgres_password
host = 'localhost'
port = '5432'
database = 'shakespeare'

engine_string = f'{dbms}+{connector}://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(engine_string)

In [ ]:
# Data
works.to_sql('works', con = engine, chunksize = 1000, if_exists = 'replace', index = False)
characters.to_sql('characters', con = engine, chunksize = 1000, if_exists = 'replace', index = False)
chapters.to_sql('chapters', con = engine, chunksize = 1000, if_exists = 'replace', index = False)
paragraphs.to_sql('paragraphs', con = engine, chunksize = 1000, if_exists = 'replace', index = False)

-36

#### Part b
Write a query to display `title`, `date`, and `totalwords` from the `works` table. Rename `date` to `year`, and sort the output by `totalwords` in descending order. Also create a new column called `era` which is equal to "early" for works created before 1600, "middle" for works created between 1600 and 1607, and "late" for works created after 1607. Finally, display only the 7th through 11th rows of the output data. [1 point]

In [ ]:
query_b = '''
SELECT
    w.title,
    w.date AS year,
    w.totalwords,
    CASE
        WHEN w.date < 1600 THEN 'early'
        WHEN w.date BETWEEN 1600 AND 1607 THEN 'middle'
        WHEN w.date > 1607 THEN 'late'
    END AS era
    FROM works w
ORDER BY w.totalwords DESC
LIMIT 5 OFFSET 6
'''

pd.read_sql_query(query_b, engine)

,title,year,totalwords,era
0,King Lear,1605,26119,middle
1,Troilus and Cressida,1601,26089,middle
2,"Henry IV, Part II",1597,25692,early
3,"Henry VI, Part II",1590,25411,early
4,The Winter's Tale,1610,24914,late


#### Part c
The `genretype` column in the "works" table designates five types of Shakespearean work:

* `t` is a tragedy, such as *Romeo and Juliet* and *Hamlet*
* `c` is a comedy, such as *A Midsummer Night's Dream* and *As You Like It*
* `h` is a history, such as *Henry V* and *Richard III*
* `s` refers to Shakespeare's sonnets
* `p` is a narrative (non-sonnet) poem, such as *Venus and Adonis* and *Passionate Pilgrim*

Write a query that generates a table that reports the average number of words in Shakepeare's works by genre type. Display the genre type and the average wordcount within genre, use appropriate aliases, and sort by the average in descending order. [1 point]

In [ ]:
query_c = '''
SELECT
    CASE
        WHEN w.genretype = 't' THEN 'tragedy'
        WHEN w.genretype = 'c' THEN 'comedy'
        WHEN w.genretype = 'h' THEN 'history'
        WHEN w.genretype = 's' THEN 'sonnets'
        WHEN w.genretype = 'p' THEN 'poem'
    END AS genre,
    AVG(w.totalwords) AS avg_words
    FROM works w
GROUP BY w.genretype
ORDER BY avg_words DESC
'''

pd.read_sql_query(query_c, engine)

,genre,avg_words
0,history,24236.000000
1,tragedy,23817.363636
2,comedy,20212.071429
3,sonnets,17515.000000
4,poem,6181.800000


#### Part d
Use a query to generate a table that contains the text of Hamlet's (the character, not just the play) longest speech, and use the `print()` function to display this text. [1 point]

In [ ]:
query_d = '''
SELECT
    char.charname AS character,
    char.charid AS character_ID,
    p.plaintext AS speech,
    p.wordcount
    FROM characters char
INNER JOIN paragraphs p
    ON char.charid = p.charid
WHERE char.charid = 'hamlet'
ORDER BY p.wordcount DESC
LIMIT 2
'''

hamlet_longest_speech = pd.read_sql_query(query_d, engine)

# Showing results
hamlet_longest_speech

,character,character_id,speech,wordcount
0,Hamlet,hamlet,"Well said, old mole! Canst work i' th' earth s...",NaN
1,Hamlet,hamlet,"Ay, so, God b' wi' ye! ...",466.0


In [ ]:
# Dropping the NaN result that was ensured to not be Hamlet's longest speech
hamlet_longest_speech = hamlet_longest_speech.dropna(subset = ['wordcount'])

# Showing results
hamlet_longest_speech

,character,character_id,speech,wordcount
1,Hamlet,hamlet,"Ay, so, God b' wi' ye! ...",466.0


In [ ]:
print(hamlet_longest_speech['speech'].iloc[0])

Ay, so, God b' wi' ye!                        [Exeunt Rosencrantz and Guildenstern
[p]Now I am alone. 
[p]O what a rogue and peasant slave am I!
[p]Is it not monstrous that this player here,
[p]But in a fiction, in a dream of passion,
[p]Could force his soul so to his own conceit
[p]That, from her working, all his visage wann'd,
[p]Tears in his eyes, distraction in's aspect,
[p]A broken voice, and his whole function suiting
[p]With forms to his conceit? And all for nothing!
[p]For Hecuba!
[p]What's Hecuba to him, or he to Hecuba,
[p]That he should weep for her? What would he do,
[p]Had he the motive and the cue for passion
[p]That I have? He would drown the stage with tears
[p]And cleave the general ear with horrid speech;
[p]Make mad the guilty and appal the free,
[p]Confound the ignorant, and amaze indeed
[p]The very faculties of eyes and ears.
[p]Yet I,
[p]A dull and muddy-mettled rascal, peak
[p]Like John-a-dreams, unpregnant of my cause, 
[p]And can say nothing! No, not for a king

### Part e
Many scenes in Shakespeare's works take place in palaces or castles. Use a query to create a table that lists all of the chapters that take place in a palace. Include the work's title, the section (renamed to "act"), the chapter (renamed to "scene"), and the description of these chapters. The setting of each scene is listed in the `description` column of the "chapters" table. [Hint: be sure to account for case sensitivity] [2 points]

In [ ]:
from sqlalchemy import text

In [ ]:
query_e = text('''
SELECT 
    w.title,
    chap.section AS act,
    chap.chapter AS scene,
    chap.description AS scene_description
    FROM chapters chap
INNER JOIN works w
    ON chap.workid = w.workid
WHERE LOWER(chap.description) LIKE '%palace%'    
''')

pd.read_sql_query(query_e, engine)

,title,act,scene,scene_description
0,Twelfth Night,2.0,4.0,DUKE ORSINO's palace.
1,Twelfth Night,1.0,4.0,DUKE ORSINO's palace.
2,Twelfth Night,1.0,1.0,DUKE ORSINO's palace.
3,All's Well That Ends Well,5.0,3.0,Rousillon. The COUNT's palace.
4,All's Well That Ends Well,5.0,2.0,Rousillon. Before the COUNT's palace.
...,...,...,...,...
120,The Winter's Tale,5.0,1.0,A room in LEONTES' palace.
121,The Winter's Tale,4.0,2.0,Bohemia. The palace of POLIXENES.
122,The Winter's Tale,2.0,3.0,A room in LEONTES' palace.
123,The Winter's Tale,2.0,1.0,A room in LEONTES' palace.


### Part f
Create a table that lists characters, the plays that the characters appear in, the number of speeches the character gives, and the average length of the speeches that the character gives. Display the character description and the work title, not the ID values. Sort the table by average speech length, and restrict the table to only those characters that give at least 20 speeches. [Hint: you will need to use a subquery.] [2 points]

In [ ]:
# Calculating number of speeches and average speech length per character
sub_query_f = '''
SELECT
    char.charname AS character,
    char.description AS character_description,
    w.title,
    COUNT(p.plaintext) AS speech_count,
    AVG(p.wordcount) AS avg_speech_length
    FROM characters char
INNER JOIN paragraphs p
    ON char.charid = p.charid
INNER JOIN works w
    ON p.workid = w.workid
GROUP BY char.charname, char.description, w.title
'''

pd.read_sql_query(sub_query_f, engine)

,character,character_description,title,speech_count,avg_speech_length
0,Aaron,"a Moor, beloved by Tamora",Titus Andronicus,57,49.719298
1,Abbot,None,Richard II,2,36.000000
2,Abhorson,an executioner,Measure for Measure,13,10.846154
3,Abraham,Servant to Montague,Romeo and Juliet,5,4.800000
4,Achilles,a Greek prince,Troilus and Cressida,74,17.824324
...,...,...,...,...,...
1326,Young Cato,friend to Brutus,Julius Caesar,3,18.000000
1327,Young Clifford,None,"Henry VI, Part II",4,91.000000
1328,Young Coriolanus,son to Coriolanus,Coriolanus,1,17.000000
1329,Young Lucius,son to Lucius,Titus Andronicus,11,32.181818


In [ ]:
query_f = f'''
SELECT *
    FROM ({sub_query_f})
WHERE speech_count >= 20
ORDER BY avg_speech_length DESC
'''

pd.read_sql_query(query_f, engine)

,character,character_description,title,speech_count,avg_speech_length
0,Poet,the voice of Shakespeare's poetry,Sonnets,154,113.733766
1,Henry IV,King of England,"Henry IV, Part I",30,85.900000
2,Poet,the voice of Shakespeare's poetry,Passionate Pilgrim,43,72.651163
3,Henry IV,King of England,"Henry IV, Part II",34,71.794118
4,King Richard II,king of England,Richard II,98,61.765306
...,...,...,...,...,...
447,(stage directions),None,Romeo and Juliet,146,3.342466
448,(stage directions),None,The Winter's Tale,67,3.238806
449,(stage directions),None,Troilus and Cressida,158,2.936709
450,(stage directions),None,Twelfth Night,108,2.935185


### Part g
Which Shakepearean works do not contain any scenes in a palace or a castle? Use a query that displays the title, genre type, and publication date of works that do not contain any scenes that take place in a palace or castle. [Hint: use your work in part e as a starting point. You will need a subquery, and you will need to think carefully about the type of join that you need to perform.][2 points]

In [ ]:
# Finding scenes that take place in a palace or castle
sub_query_g = text('''
SELECT
    w.workid,
    w.title,
    chap.description AS scene_description
    FROM chapters chap
INNER JOIN works w
    ON chap.workid = w.workid
WHERE LOWER(chap.description) LIKE '%palace%'
    OR LOWER(chap.description) LIKE '%castle%'    
''')

pd.read_sql_query(sub_query_g, engine)

,workid,title,scene_description
0,12night,Twelfth Night,DUKE ORSINO's palace.
1,12night,Twelfth Night,DUKE ORSINO's palace.
2,12night,Twelfth Night,DUKE ORSINO's palace.
3,allswell,All's Well That Ends Well,Rousillon. The COUNT's palace.
4,allswell,All's Well That Ends Well,Rousillon. Before the COUNT's palace.
...,...,...,...
181,winterstale,The Winter's Tale,A room in LEONTES' palace.
182,winterstale,The Winter's Tale,Bohemia. The palace of POLIXENES.
183,winterstale,The Winter's Tale,A room in LEONTES' palace.
184,winterstale,The Winter's Tale,A room in LEONTES' palace.


In [ ]:
query_g = text(f'''
SELECT
    w.workid,
    w.title,
    chap.description AS scene_description
    FROM works w
INNER JOIN chapters chap
    ON w.workid = chap.workid
LEFT JOIN ({sub_query_g}) AS s
    ON w.workid = s.workid
WHERE s.workid IS NULL
''')

pd.read_sql_query(query_g, engine)

,workid,title,scene_description
0,coriolanus,Coriolanus,Rome. A street.
1,coriolanus,Coriolanus,Corioli. The Senate-house.
2,coriolanus,Coriolanus,Rome. A room in CORIOLANUS' house.
3,coriolanus,Coriolanus,Before Corioli.
4,coriolanus,Coriolanus,Corioli. A street.
...,...,...,...
358,timonathens,Timon of Athens,Before the walls of Athens.
359,timonathens,Timon of Athens,"The woods. Timon's cave, and a rude tomb seen."
360,timonathens,Timon of Athens,Before the walls of Athens.
361,venusadonis,Venus and Adonis,Dedication.


### Problem 2
The following file contains JSON formatted data of the official English-language translations of every constitution currently in effect in the world:

In [9]:
const = requests.get("https://github.com/jkropko/DS-6001/raw/master/localdata/const.json")
const_json = json.loads(const.text)
pd.DataFrame.from_records(const_json)

,text,country,adopted,revised,reinstated,democracy
0,'Afghanistan 2004 Preamble \n﻿In the na...,Afghanistan,2004,NaN,NaN,0.372201
1,'Albania 1998 (rev. 2012) Preamble \nWe...,Albania,1998,2012.0,NaN,0.535111
2,'Andorra 1993 Preamble \nThe Andorran P...,Andorra,1993,NaN,NaN,NaN
3,"'Angola 2010 Preamble \nWe, the people ...",Angola,2010,NaN,NaN,0.315043
4,'Antigua and Barbuda 1981 Preamble \nWH...,Antigua and Barbuda,1981,NaN,NaN,NaN
...,...,...,...,...,...,...
140,'Uzbekistan 1992 (rev. 2011) Preamble \...,Uzbekistan,1992,2011.0,NaN,0.195932
141,'Viet Nam 1992 (rev. 2013) Preamble \nI...,Viet Nam,1992,2013.0,NaN,0.251461
142,'Yemen 1991 (rev. 2001) PART ONE. THE FOUN...,Yemen,1991,2001.0,NaN,0.125708
143,"'Zambia 1991 (rev. 2009) Preamble \nWE,...",Zambia,1991,2009.0,NaN,0.405497


The text of the constitutions are available from the [Wolfram Data Repository](https://datarepository.wolframcloud.com/resources/World-Constitutions). I also included scores that represent the level of democractic quality in each country as of 2016. These scores are compiled by the [Varieties of Democracy (V-Dem)](https://www.v-dem.net/en/) project. Higher scores indicate greater levels of democratic openness and competition.

#### Part a
Connect to your local MongoDB server and create a new collection for the constitution data. Use `.delete_many({})` to remove any existing data from this collection, and insert the data in `const_json` into this collection. [2 points]

In [12]:
# Environmental variables
mongo_username = os.getenv('MONGO_INITDB_ROOT_USERNAME')
mongo_password = os.getenv('MONGO_INITDB_ROOT_PASSWORD')
mongo_host = os.getenv('MONGO_HOST', 'localhost')
mongo_port = os.getenv('MONGO_PORT', '27017')

mongo_url = f"mongodb://{mongo_username}:{mongo_password}@{mongo_host}:{mongo_port}/"

In [13]:
# Connect to MongoDB
client = pymongo.MongoClient(mongo_url)
const = client["const"]
constcollection = const["constcollection"]

In [15]:
# Deleting existing data
constcollection.delete_many({})

# Inserting new data
constcollection.insert_many(const_json)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 669d7543ff691eb33ea473f0, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

#### Part b
Use MongoDB queries and the `dumps()` and `loads()` functions from the `bson` package to produce dataframes with the following restrictions:

* The country, adoption year, and democracy features (and not `_id`, text, revised, or reinstated) for countries with constitutions that were written after 1990 
* The country, adoption year, and democracy features (and not `_id`, text, revised, or reinstated) for countries with constitutions that were written after 1990 AND have a democracy score of less than 0.5
* The country, adoption year, and democracy features (and not `_id`, text, revised, or reinstated) for countries with constitutions that were written after 1990 OR have a democracy score of less than 0.5

[1 point]

In [16]:
# Query 1: country, adoption year, and democracy features after 1990
query1 = {'adoption':{'$gt':1990}}
projection = {'_id': 0, 'country': 1, 'adoption': 1, 'democracy': 1}

mongo_read_query = (constcollection, query1)

In [18]:
# Query 1 DataFrame
q1 = dumps(constcollection.find(query1, projection))
q1_data = loads(q1)
q1_df = pd.DataFrame(q1_data)

q1_df

In [ ]:
# Query 2: country, adoption year, and democracy features after 1990 and democracy score < 0.5
query2 = {'adoption':{'$gt':1990}, 'democracy':{'$lt':0.5}}
projection = {'_id': 0, 'country': 1, 'adoption': 1, 'democracy': 1}

mongo_read_query = (constcollection, query2)

In [ ]:
# Query 2 DataFrame
q2 = dumps(constcollection.find(query2, projection))
q2_data = loads(q2)
q2_df = pd.DataFrame(q2_data)

q2_df

In [ ]:
# Query 3: country, adoption year, and democracy features after 1990 or democracy score < 0.5
query3 = {'$or': [{'adoption':{'$gt':1990}, 'democracy':{'$lt':0.5}}]}
projection = {'_id': 0, 'country': 1, 'adoption': 1, 'democracy': 1}

mongo_read_query = (constcollection, query3)

In [ ]:
# Query 3 DataFrame
q3 = dumps(constcollection.find(query3, projection))
q3_data = loads(q3)
q3_df = pd.DataFrame(q3_data)

q3_df

#### Part c
According to the Varieties of Democracy project, [Hungary has become less democratic](https://www.v-dem.net/en/news/democratic-declines-hungary/) over the last few years, and can no longer be considered a democracy. Update the record for Hungary to set the democracy score at 0.4. Then query the database to extract the record for Hungary and display the data in a dataframe. [1 point]

In [ ]:
# Original Hungary democracy score
hungary_original = constcollection.find_one({'country':'Hungary'}, {'_id': 0, 'democracy': 1})

hungary_original

In [ ]:
# Setting Hungary democracy score = 0.4
constcollection.update_one({'country':'Hungary'}, {'$set':{'democracy': 0.4}})
hungary_now = constcollection.find_one({'country':'Hungary'}, {'_id': 0, 'country': 1, 'adoption': 1, 'democracy': 1})

hungary_now

In [ ]:
# Showing results
hungary_df = pd.DataFrame([hungary_now])

hungary_df

#### Part d
Set the `text` field in the database as a text index. Then query the database to find all constitutions that contain the exact phrase "freedom of speech". Display the country name, adoption year, and democracy scores in a dataframe for the constitutions that match this query. [2 points]

In [ ]:
# Text index
constcollection.create_index([('text', 'text')])

In [ ]:
# 'Freedom of speech' (fos) query
fos_query = {'$text':{'$search':'\'freedom of speech\''}}
projection = {'_id': 0, 'country': 1, 'adoption': 1, 'democracy': 1}

In [ ]:
# Showing results
fos_data = dumps(list(constcollection.find(fos_query, projection)))
data = loads(fos_data)
fos_df = pd.DataFrame(data)

fos_df

#### Part e
Use a query to search for the terms "freedom", "liberty", "legal", "justice", and "rights". Generate a text score for all of the countries, and display the data for the countries with the top 10 relevancy scores in a dataframe. [2 points]

In [ ]:
# Querying 'freedom, liberty, legal, justice, rights'
query = {'$text':{'$search':'freedom liberty legal justice rights'}}
projection = {'_id': 0, 'country': 1, 'adoption': 1, 'democracy': 1, 'score':{'$meta':'textScore'}}

# Text score
query_data = dumps(list(constcollection.find(query, projection).sort([('score', {'$meta':'textScore'})])))

In [ ]:
# Showing results
data = loads(query_data)
df = pd.DataFrame(data)

df.head(10)

### Question 3
Close the connections to the PostgreSQL and MongoDB databases. [1 point]

In [ ]:
# Committing and closing PostgreSQL
dbserver.commit()
dbserver.close()

In [ ]:
# Closing MongoDB
client.close()